# Imports

In [1]:
!pip install -i https://pypi.clarin-pl.eu lpmn_client

Looking in indexes: https://pypi.clarin-pl.eu


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import re, string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from lpmn_client.src.requester import Requester
import zipfile
from tensorflow.keras.utils import to_categorical
import xml.etree.ElementTree as ET




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data loading

In [3]:
data_path = '/content/drive/MyDrive/ZIwG P/citations.csv'
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 120)

df = pd.read_csv(data_path)
df = df.dropna()
df = df.reset_index(drop=True)
# df.info()

classes_names = {0: 'Fałsz', 1: 'Prawda', 2: "Manipulacja", 3: "Nieweryfikowalne"}
df['label'].replace({'Fałsz': 0, 'Prawda': 1, 'Manipulacja': 2, 'Nieweryfikowalne': 3}, inplace=True)
df = df[(df['label'] == 0) | (df['label'] == 1)]
# X.head()
# y.head(10)
df

,content,author,label
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0
...,...,...,...
4330,"Chroniliście SKOK-i, chroniliście swoich kolesi. Dlatego z budżetu państwa ponad 4,5 mld zł trzeba było im wypłacić.",Borys Budka,1
4332,"Jeżeli chodzi o penalizację homoseksualizmu, to w Polsce nigdy czegoś takiego nie było. Ani za I Rzeczypospolitej, a...",Kosma Złotowski,0
4333,Rządowy Fundusz Inwestycji Lokalnych na Pomorzu – największy beneficjent gmina miasta Gdańsk.,Marcin Horała,1
4334,"Zarówno WHO, EMA, czyli Europejska Agencja Leków, jak i polskie instytucje, w tym Urząd Rejestracji Produktów Leczni...",Michał Dworczyk,1


# Text preprocessing

In [4]:
def remove_punctuation(text):
    no_punctuation_text = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation_text

def lemmatize(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|wcrft2({"guesser":false, "morfeusz2":true})'

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [id.text for id in string_ids])
    requester.download_response(response[0], './lem.zip')

    archive = zipfile.ZipFile('lem.zip', 'r')
    data = archive.read(archive.namelist()[0])

    text = [word.text for word in ET.fromstring(data).findall('chunk/sentence/tok/lex/base')]
    return text


def tokenize(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text

def remove_stopwords(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|morphoDita|dir|termopl2({\"mw\":false,\"sw\":\"/resources/termopl/termopl_sw.txt\",' \
                 '\"cp\":\"/resources/termopl/termopl_cp.txt\"}) '

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [string_id.text for string_id in string_ids])
    requester.download_response(response[0], './no_stopwords.zip')

    with zipfile.ZipFile('no_stopwords.zip', 'r') as archive:
        with archive.open(archive.namelist()[0]) as data:
            column_names = ['idx', 'ranking', 'output_phrase', 'original_phrase', 'c-value',
                            'length', 'freq_s', 'freq_in', 'context']
            df = pd.read_csv(data, sep='\t', names=column_names)

    lemmatized_text_without_stopwords = df['output_phrase'].tolist()
    return lemmatized_text_without_stopwords


#TODO when we decide what how to preprocess text
# now it is just copy-paste from online course
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text


In [5]:
# Took only 5 first elements to demonstrate the output
df['lemmatized'] = df['content'][:5].apply(lambda x: lemmatize(x))
df.head()


Processing query complete, file id /requests/makezip/dfd53cb3-a498-4e53-a787-b2860bf325a0

Processing query complete, file id /requests/makezip/681f200d-9ffb-444b-963b-0808b4f50fd7

Processing query complete, file id /requests/makezip/f997cfac-4c23-4249-b3a1-e0003ebfdc7f

Processing query complete, file id /requests/makezip/fcfcce65-6c82-46b5-9b90-8428dc58feeb

Processing query complete, file id /requests/makezip/3aa1e7d7-d4ff-4921-8a58-5a9728521d98


,content,author,label,lemmatized
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0,"[o, ile, my, podnieść, być, pensja, minimalny, w, czas, 8, lato, ?, (, …, ), o, ponad, 90, proca, .]"
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1,"[w, ranking, praworządność, World, Justice, Project, w, 2020, r, ., Polska, znów, spadły, na, niski, pozycja, .]"
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1,"[na, uniwersytet, Warszawskie, powstać, taki, raport, ,, który, być, oczywiście, państwo, znany, ,, z, który, wyraźn..."
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1,"[średnia, emerytura, w, Polska, kobieta, i, mężczyzna, różny, się, aż, o, 1000, złoty, .]"
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0,"[prosić, mi, pokazać, (, …, ), jeden, osoba, skazana, przez, trybunał, stan, od, czas, ,, kiedy, trybunał, stan, w, ..."


In [6]:
df['lemmatized_without_stopwords'] = df['content'][:5].apply(lambda x: remove_stopwords(x))
df.head()

Processing query in progress... |----------------------------------------------------------------------------------------------------| 0.0% percent complete
Processing query complete, file id /requests/makezip/08622dfe-77d3-4d1b-8526-d8e95a04fca8

Processing query complete, file id /requests/makezip/848ace14-d7df-4c38-80f9-47693161650c

Processing query complete, file id /requests/makezip/cce20e30-df11-415f-9f8a-02131efb3636

Processing query complete, file id /requests/makezip/c111a862-ad3b-48b0-956c-e22fa8cfc4c4

Processing query complete, file id /requests/makezip/2a4d4eef-9736-4e01-9599-79d1188feb22


,content,author,label,lemmatized,lemmatized_without_stopwords
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0,"[o, ile, my, podnieść, być, pensja, minimalny, w, czas, 8, lato, ?, (, …, ), o, ponad, 90, proca, .]","[pensja minimalny, pensja]"
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1,"[w, ranking, praworządność, World, Justice, Project, w, 2020, r, ., Polska, znów, spadły, na, niski, pozycja, .]","[niski pozycja, ranking praworządność, Project, Polska, praworządność, ranking, pozycja]"
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1,"[na, uniwersytet, Warszawskie, powstać, taki, raport, ,, który, być, oczywiście, państwo, znany, ,, z, który, wyraźn...","[spółka skarb państwo, uniwersytet warszawski, spory kwota, szczególny preferencja, reklama medialny, zamówienie rek..."
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1,"[średnia, emerytura, w, Polska, kobieta, i, mężczyzna, różny, się, aż, o, 1000, złoty, .]","[Polska kobieta, średni emerytura, mężczyzna, emerytura, kobieta, Polska]"
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0,"[prosić, mi, pokazać, (, …, ), jeden, osoba, skazana, przez, trybunał, stan, od, czas, ,, kiedy, trybunał, stan, w, ...","[trybunał stan, jeden osoba skazać, urzędnik państwowy, człowiek, czas, Polska, korupcja, malwersacja, jeden osoba, ..."


In [7]:
df['clean_text'] = df['content'].apply(lambda x: clean_text(x))
df.head()

,content,author,label,lemmatized,lemmatized_without_stopwords,clean_text
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0,"[o, ile, my, podnieść, być, pensja, minimalny, w, czas, 8, lato, ?, (, …, ), o, ponad, 90, proca, .]","[pensja minimalny, pensja]","[o, ile, my, podnieśliśmy, pensję, minimalną, w, czasie, 8, lat, o, ponad, 90, proc]"
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1,"[w, ranking, praworządność, World, Justice, Project, w, 2020, r, ., Polska, znów, spadły, na, niski, pozycja, .]","[niski pozycja, ranking praworządność, Project, Polska, praworządność, ranking, pozycja]","[w, rankingu, praworządności, world, justice, project, w, 2020, r, polska, znów, spadła, na, niższą, pozycję]"
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1,"[na, uniwersytet, Warszawskie, powstać, taki, raport, ,, który, być, oczywiście, państwo, znany, ,, z, który, wyraźn...","[spółka skarb państwo, uniwersytet warszawski, spory kwota, szczególny preferencja, reklama medialny, zamówienie rek...","[na, uniwersytecie, warszawskim, powstał, taki, raport, który, jest, oczywiście, państwu, znany, z, którego, wyraźni..."
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1,"[średnia, emerytura, w, Polska, kobieta, i, mężczyzna, różny, się, aż, o, 1000, złoty, .]","[Polska kobieta, średni emerytura, mężczyzna, emerytura, kobieta, Polska]","[średnia, emerytura, w, polsce, kobiet, i, mężczyzn, różni, się, aż, o, 1000, zł]"
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0,"[prosić, mi, pokazać, (, …, ), jeden, osoba, skazana, przez, trybunał, stan, od, czas, ,, kiedy, trybunał, stan, w, ...","[trybunał stan, jeden osoba skazać, urzędnik państwowy, człowiek, czas, Polska, korupcja, malwersacja, jeden osoba, ...","[proszę, mi, pokazać, jedną, osobę, skazaną, przez, trybunał, stanu, od, czasu, kiedy, trybunał, stanu, w, polsce, w..."


# Feature creation

In [8]:
def count_uppercase_letters(text):
    count = sum([1 for char in text if char.isupper()])
    return (count / (len(text) - text.count(' '))) * 100


def count_exclamation_marks(text):
    count = text.count('!')
    return (count / (len(text) - text.count(' '))) * 100


def count_question_marks(text):
    count = text.count('?')
    return (count / (len(text) - text.count(' '))) * 100


def count_quotation_marks(text):
    count = text.count('"')
    return (count / (len(text) - text.count(' '))) * 100


def count_punctuation(text):
    count = sum([1 for char in text if char in string.punctuation])
    return (count / (len(text) - text.count(' '))) * 100


def count_text_length(text):
    return len(text) - text.count(' ')


def get_sentiment(text):
    requester = Requester('241393@student.pwr.edu.pl')
    lpmn_query = 'any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({' \
                 '"lang":"polish", "split_paragraphs": false})'

    string_ids = requester.upload_strings([text])
    response = requester.process_query(lpmn_query, [string_id.text for string_id in string_ids])
    requester.download_response(response[0], './sentiment.zip')

    with zipfile.ZipFile('sentiment.zip', 'r') as archive:
        with archive.open(archive.namelist()[0]) as data:
            df = pd.read_csv(data, sep=';')

    sentiment_value = sum([int(entry) for entry in df['Polarity'].values if entry != 'None'])
    return sentiment_value


def count_positive_words():
    with zipfile.ZipFile('sentiment.zip', 'r') as archive:
        with archive.open(archive.namelist()[0]) as data:
            df = pd.read_csv(data, sep=';')

    positive_words = sum([1 for entry in df['Polarity'].values if entry != 'None' and int(entry) > 0])
    return (positive_words / len(df['Polarity'])) * 100


def count_negative_words():
    with zipfile.ZipFile('sentiment.zip', 'r') as archive:
        with archive.open(archive.namelist()[0]) as data:
            df = pd.read_csv(data, sep=';')

    negative_words = sum([1 for entry in df['Polarity'].values if entry != 'None' and int(entry) < 0])
    return (negative_words / len(df['Polarity'])) * 100


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.25)

In [10]:
X_train

3866    [na, 100, szwajcarów, przypada, około, 24, sztuki, broni, w, tym, automatycznej, w, polsce, 15, sztuk, broni, na, 10...
2107                                                                    [a, najwięcej, dostała, platforma, 4159, w, roku, 2007]
1392    [w, programie, pawła, adamowicza, obiecany, jest, wzrost, o, 50, środków, na, remont, i, utrzymanie, chodników, a, w...
975      [obciążenie, prokuratora, wojskowego, było, około, 15, razy, mniejsze, niż, prokuratora, w, prokuraturze, powszechnej]
3675    [, już, 3, raz, z, rzędu, w, 3, kadencji, już, otrzymałem, najlepszy, wynik, w, kraju, tym, razem, to, jest, 77, tys...
                                                                 ...                                                           
2309    [na, ile, się, znam, na, ordynacji, wyborczej, to, ona, zakazuje, po, pierwsze, finansowania, kampanii, przez, podmi...
102     [tu, w, procesie, przyznawania, pomocy, finansowej, z, funduszy, wsparcia, kultury, przyp, demag

## Vectorization

### N-gram

In [11]:
# range = (2, 2)

# ngram_vect = CountVectorizer(ngram_range=range)
# X_ngram = ngram_vect.fit_transform(X['content'])

# X_ngram_df = pd.DataFrame(X_ngram.toarray())
# X_ngram_df.columns = ngram_vect.get_feature_names()
# X_ngram_df

### TF-IDF

In [12]:
# tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(X_train['content'])

# X_train_vect = tfidf_vect.transform(X_train['content'])
# X_test_vect = tfidf_vect.transform(X_test['content'])

In [13]:
# X_test_vect[0].toarray()

# Model nr 1

# Model nr 2

# Deep Learning

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq_pad = pad_sequences(X_train_seq, len(max(X_train_seq, key=len)))
X_test_seq_pad = pad_sequences(X_test_seq, len(max(X_train_seq, key=len)))



In [15]:
X_train_seq_pad.shape

(2676, 177)

## RNN model

In [16]:
batch_size = 64

model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 64))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, recurrent_initializer='glorot_uniform'))
# model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(4, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          813760    
_________________________________________________________________
lstm (LSTM)                  (None, None, 32)          12416     
_________________________________________________________________
dense (Dense)                (None, None, 1)           33        
Total params: 826,209
Trainable params: 826,209
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
early_stopping = EarlyStopping(monitor='loss', patience=3)

# y_train_cat = to_categorical(y_train, 4)
# y_test_cat = to_categorical(y_test, 4)

history = model.fit(X_train_seq_pad, y_train, validation_data=(X_test_seq_pad, y_test), batch_size=batch_size, epochs=32, callbacks=[early_stopping])

Epoch 1/32
42/42 [==============================] - 13s 249ms/step - loss: 0.6482 - accuracy: 0.6864 - val_loss: 0.6075 - val_accuracy: 0.7077
Epoch 2/32
29/42 [===================>..........] - ETA: 2s - loss: 0.5756 - accuracy: 0.7428

KeyboardInterrupt: ignored

In [ ]:
prediction = model.predict_classes(X_test_seq_pad[48])
# len(X_test_seq_pad[48])
print(len(prediction))
prediction = int(stats.mode(prediction).mode)
print('Treść wypowiedzi:')
print(X_test.iloc[48])
print('Klasyfikacja: ', classes_names[prediction])
# predictions = np.argmax(model.predict(test_image_gen), axis=-1)

# plt.figure(figsize=(10,6))
# sns.heatmap(confusion_matrix(test_image_gen.classes,predictions),annot=True)

# Evaluation